# deepfake img preprocessing

# Face Detection & Cropping using YOLO (Best for Real-Time Video Processing)

In [1]:
pip install opencv-python ultralytics

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install albumentations

In [1]:
import cv2
import os
import torch
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
from ultralytics import YOLO

# Load YOLOv8 face detection model
model = YOLO("yolov8n-face.pt")  # Ensure you have this model

# Directories
datasets = ["train", "val", "test"]  # Process all datasets
batch_size = 32  # Adjust based on GPU memory
IMG_SIZE = (224, 224)  # Target image size

# Albumentations Augmentation Pipeline (You can tweak this)
augmentation_pipeline = A.Compose([
    A.HorizontalFlip(p=0.5),         # Random horizontal flip
    A.Rotate(limit=15, p=0.5),        # Rotate ±15 degrees
    ToTensorV2()                      # Convert to Tensor (optional)
])

def detect_and_process(image_path, save_path):
    """Detects faces, crops, resizes, converts to grayscale, normalizes, and augments."""
    image = cv2.imread(image_path)
    if image is None:
        return

    results = model(image)  # YOLO Face Detection
    boxes = results[0].boxes.xyxy  # Extract bounding boxes

    # If a face is detected, crop the first one
    if len(boxes) > 0:
        x1, y1, x2, y2 = map(int, boxes[0])  
        cropped_face = image[y1:y2, x1:x2]
    else:
        cropped_face = image  # If no face detected, keep full image

    # Resize to standard size
    resized_face = cv2.resize(cropped_face, IMG_SIZE)

    # Convert to Grayscale
    gray_face = cv2.cvtColor(resized_face, cv2.COLOR_BGR2GRAY)

    # Normalize Pixel Values (0 to 1)
    normalized_face = gray_face / 255.0

    # Convert back to 3-channel format (needed for augmentation)
    normalized_face = np.stack([normalized_face] * 3, axis=-1)

    # Apply Data Augmentation (Optional)
    # augmented = augmentation_pipeline(image=normalized_face)["image"]
    augmented = normalized_face
    
    # Save preprocessed image
    cv2.imwrite(save_path, (augmented * 255).astype(np.uint8))  # Convert back to image format

# Batch Processing Loop
for dataset in datasets:
    input_dir = f"D:/Singularity101/dataset-new-n/{dataset}"
    output_dir = f"D:/Singularity101/processed-dataset-new-n/preprocessed_{dataset}"
    os.makedirs(output_dir, exist_ok=True)

    for category in ["real", "fake"]:
        input_folder = os.path.join(input_dir, category)
        output_folder = os.path.join(output_dir, category)
        os.makedirs(output_folder, exist_ok=True)

        images = [img for img in os.listdir(input_folder) if img.endswith(('.jpg', '.png', '.jpeg'))]

        for i in range(0, len(images), batch_size):
            batch = images[i:i+batch_size]  # Process in batches
            for img_name in batch:
                img_path = os.path.join(input_folder, img_name)
                save_path = os.path.join(output_folder, img_name)
                detect_and_process(img_path, save_path)

            print(f"✅ Processed {i+batch_size}/{len(images)} images in {dataset}/{category}")

print("✅ Preprocessing completed!")


0: 640x640 (no detections), 62.5ms
Speed: 31.2ms preprocess, 62.5ms inference, 140.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 68.9ms
Speed: 11.6ms preprocess, 68.9ms inference, 9.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 55.8ms
Speed: 15.7ms preprocess, 55.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 81.8ms
Speed: 12.6ms preprocess, 81.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 60.7ms
Speed: 16.6ms preprocess, 60.7ms inference, 4.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 57.1ms
Speed: 20.0ms preprocess, 57.1ms inference, 7.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 55.8ms
Speed: 16.9ms preprocess, 55.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 54.3ms
Speed: 17.9ms preprocess

# deepfake video preprocessing

In [7]:
pip install opencv-python

In [9]:
import os
import cv2

def extract_frames(video_path, output_folder, num_frames=10):
    """Extract exactly `num_frames` frames evenly spaced from a video."""
    os.makedirs(output_folder, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total_frames < num_frames:
        print(f"Warning: {video_path} has only {total_frames} frames, extracting all available frames.")

    frame_indices = [int(i * total_frames / num_frames) for i in range(num_frames)]  # Evenly spaced frames
    frame_idx = 1
    frame_count = 0

    while True:
        success, frame = cap.read()
        if not success:
            break

        if frame_count in frame_indices:
            frame_filename = os.path.join(output_folder, f"frame_{frame_idx:03d}.jpg")
            cv2.imwrite(frame_filename, frame)
            frame_idx += 1

        frame_count += 1

    cap.release()

def process_videos(video_folder, output_root):
    """Process videos in a folder and extract frames into structured directories."""
    class_name = os.path.basename(video_folder)  # Get 'Fake' or 'Real'
    output_class_folder = os.path.join(output_root, class_name)
    os.makedirs(output_class_folder, exist_ok=True)

    for idx, video_file in enumerate(os.listdir(video_folder)):
        if video_file.endswith((".mp4", ".avi", ".mov")):
            video_path = os.path.join(video_folder, video_file)
            video_output_folder = os.path.join(output_class_folder, f"video_{idx+1}")
            extract_frames(video_path, video_output_folder, num_frames=10)  # Ensure 10 frames per video

def generate_dataset_structure(fake_videos, real_videos, dataset_root="dataset-video-n"):
    """Generate the dataset folder structure from fake and real videos."""
    os.makedirs(dataset_root, exist_ok=True)
    
    process_videos(fake_videos, dataset_root)  # Process Fake videos
    process_videos(real_videos, dataset_root)  # Process Real videos

# Example usage
fake_videos_folder = "D:/Singularity101/dataset-raw-n/Fake"
real_videos_folder = "D:/Singularity101/dataset-raw-n/Real"

generate_dataset_structure(fake_videos_folder, real_videos_folder)

In [8]:
import os
import shutil
import random

def split_dataset(dataset_path, output_path, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    """
    Splits dataset into train, validation, and test sets.
    """
    assert train_ratio + val_ratio + test_ratio == 1, "Ratios must sum to 1!"

    categories = ["Fake", "Real"]
    os.makedirs(output_path, exist_ok=True)

    for split in ["train", "val", "test"]:
        for category in categories:
            os.makedirs(os.path.join(output_path, split, category), exist_ok=True)

    for category in categories:
        class_path = os.path.join(dataset_path, category)
        videos = sorted(os.listdir(class_path))  # Get video folders

        # Shuffle and split dataset
        random.shuffle(videos)
        train_cutoff = int(len(videos) * train_ratio)
        val_cutoff = train_cutoff + int(len(videos) * val_ratio)

        train_videos = videos[:train_cutoff]
        val_videos = videos[train_cutoff:val_cutoff]
        test_videos = videos[val_cutoff:]

        for split, video_list in zip(["train", "val", "test"], [train_videos, val_videos, test_videos]):
            for video in video_list:
                src = os.path.join(class_path, video)
                dest = os.path.join(output_path, split, category, video)
                shutil.copytree(src, dest)

    print("Dataset successfully split into train, val, and test sets!")

# Example usage
dataset_path = "D:/Singularity101/dataset-video-n"  # Path to your current dataset
output_path = "dataset-new-video-n"  # Path to save train/val/test
split_dataset(dataset_path, output_path)

Dataset successfully split into train, val, and test sets!


In [10]:
import os
import numpy as np
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image

# Load pre-trained ResNet-50 model (without the last FC layer)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet50 = models.resnet50(pretrained=True)
resnet50 = torch.nn.Sequential(*list(resnet50.children())[:-1])  # Remove FC layer
resnet50.to(device)
resnet50.eval()

# Define Image Preprocessing (Resize, Normalize)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def extract_features(image_path):
    """Extract ResNet-50 features from an image."""
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension
    with torch.no_grad():
        features = resnet50(image)  # Extract features
    return features.squeeze().cpu().numpy()  # Convert to NumPy array

def process_dataset(dataset_root, output_root):
    """Extract and save features for train, val, and test sets."""
    os.makedirs(output_root, exist_ok=True)

    for split in ["train", "val", "test"]:
        for category in ["Fake", "Real"]:
            input_folder = os.path.join(dataset_root, split, category)
            output_folder = os.path.join(output_root, split, category)
            os.makedirs(output_folder, exist_ok=True)

            for video in os.listdir(input_folder):
                video_path = os.path.join(input_folder, video)
                video_features = []

                for frame in sorted(os.listdir(video_path)):  # Process frames in order
                    frame_path = os.path.join(video_path, frame)
                    if frame.endswith(".jpg"):
                        features = extract_features(frame_path)
                        video_features.append(features)

                # Save features as a NumPy array
                np.save(os.path.join(output_folder, f"{video}.npy"), np.array(video_features))

    print("Feature extraction complete!")

# Run the feature extraction
dataset_root = "D:/Singularity101/dataset-new-video-n"
output_root = "processed-dataset-new-video-n"  # New directory to store extracted features
process_dataset(dataset_root, output_root)

C:\Users\anshu\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\anshu\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Feature extraction complete!


In [2]:
import os
from PIL import Image
import torchvision.transforms as transforms

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor()
])

def preprocess_and_save(dataset_root, output_root):
    """
    Preprocess frames in 'train', 'val', and 'test' folders and save them while keeping the same structure.
    """
    for split in ["train", "val", "test"]:
        input_split_folder = os.path.join(dataset_root, split)
        output_split_folder = os.path.join(output_root, split)

        if not os.path.exists(input_split_folder):
            continue

        for class_name in ["Fake", "Real"]:
            input_class_folder = os.path.join(input_split_folder, class_name)
            output_class_folder = os.path.join(output_split_folder, class_name)

            if not os.path.exists(input_class_folder):
                continue
            os.makedirs(output_class_folder, exist_ok=True)

            for video_folder in os.listdir(input_class_folder):
                input_video_folder = os.path.join(input_class_folder, video_folder)
                output_video_folder = os.path.join(output_class_folder, video_folder)

                if os.path.isdir(input_video_folder):
                    os.makedirs(output_video_folder, exist_ok=True)

                    for frame in sorted(os.listdir(input_video_folder)):
                        if frame.endswith(".jpg") or frame.endswith(".png"):
                            input_frame_path = os.path.join(input_video_folder, frame)
                            output_frame_path = os.path.join(output_video_folder, frame)

                            # Open and process the image
                            image = Image.open(input_frame_path).convert("RGB")  # Ensure RGB format
                            image = transform(image)  # Apply transformations
                            image = transforms.ToPILImage()(image)  # Convert back to image
                            image.save(output_frame_path)  # Save preprocessed image

                            print(f"Processed: {output_frame_path}")

# Example usage
dataset_root = "dataset-new-video-n"  # Original dataset folder
output_root = "Dataset_Preprocessed"  # Folder to store preprocessed frames
preprocess_and_save(dataset_root, output_root)

Processed: Dataset_Preprocessed\train\Fake\video_1\frame_001.jpg
Processed: Dataset_Preprocessed\train\Fake\video_1\frame_002.jpg
Processed: Dataset_Preprocessed\train\Fake\video_1\frame_003.jpg
Processed: Dataset_Preprocessed\train\Fake\video_1\frame_004.jpg
Processed: Dataset_Preprocessed\train\Fake\video_1\frame_005.jpg
Processed: Dataset_Preprocessed\train\Fake\video_1\frame_006.jpg
Processed: Dataset_Preprocessed\train\Fake\video_1\frame_007.jpg
Processed: Dataset_Preprocessed\train\Fake\video_1\frame_008.jpg
Processed: Dataset_Preprocessed\train\Fake\video_1\frame_009.jpg
Processed: Dataset_Preprocessed\train\Fake\video_1\frame_010.jpg
Processed: Dataset_Preprocessed\train\Fake\video_10\frame_001.jpg
Processed: Dataset_Preprocessed\train\Fake\video_10\frame_002.jpg
Processed: Dataset_Preprocessed\train\Fake\video_10\frame_003.jpg
Processed: Dataset_Preprocessed\train\Fake\video_10\frame_004.jpg
Processed: Dataset_Preprocessed\train\Fake\video_10\frame_005.jpg
Processed: Dataset_P